### 3.3.1. 生成数据集

In [21]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

### 3.3.2. 读取数据集

In [22]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

### 3.3.3. 定义模型

In [23]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

### 3.3.4. 初始化模型参数

In [24]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

### 3.3.5. 定义损失函数

In [25]:
""" loss = nn.MSELoss() """
""" loss = nn.MSELoss(reduction='sum') """
loss = nn.HuberLoss()

### 3.3.6. 定义优化算法

In [34]:
""" trainer = torch.optim.SGD(net.parameters(), lr=0.03) """
trainer = torch.optim.SGD(net.parameters(), lr=0.03) # if nn.MSELoss(reduction='sum')

### 3.3.7 训练

In [39]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        for param in net.parameters():
            print(param.grad)
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

tensor([[-0.0018,  0.0010]])
tensor([0.0026])
tensor([[ 0.0055, -0.0025]])
tensor([0.0014])
tensor([[-0.0036,  0.0007]])
tensor([-0.0012])
tensor([[ 0.0012, -0.0023]])
tensor([-0.0023])
tensor([[-0.0030,  0.0003]])
tensor([0.0017])
tensor([[ 0.0013, -0.0031]])
tensor([-0.0023])
tensor([[0.0045, 0.0034]])
tensor([-0.0010])
tensor([[-0.0043, -0.0030]])
tensor([0.0060])
tensor([[-6.0234e-05, -2.5485e-03]])
tensor([-0.0033])
tensor([[-0.0007,  0.0002]])
tensor([-0.0049])
tensor([[0.0013, 0.0043]])
tensor([-0.0059])
tensor([[0.0010, 0.0072]])
tensor([-0.0036])
tensor([[0.0004, 0.0027]])
tensor([0.0015])
tensor([[-0.0031,  0.0005]])
tensor([0.0029])
tensor([[-0.0043,  0.0066]])
tensor([0.0030])
tensor([[-0.0033, -0.0043]])
tensor([0.0012])
tensor([[ 0.0012, -0.0028]])
tensor([0.0036])
tensor([[ 0.0011, -0.0038]])
tensor([-0.0004])
tensor([[ 0.0024, -0.0006]])
tensor([0.0037])
tensor([[ 0.0020, -0.0033]])
tensor([-4.6718e-05])
tensor([[ 0.0006, -0.0018]])
tensor([-0.0016])
tensor([[ 0.0024, -

### Practice
#### 问题解析和解答

---

##### **1. 如果将小批量的总损失替换为小批量损失的平均值，需要如何更改学习率？**

**答：**

将小批量的总损失替换为小批量损失的平均值时，梯度的大小会被缩小到原来的 $1 / \text{batch\_size}$。为了保持优化过程的稳定性，学习率通常需要相应地调整。

- 如果框架没有自动归一化处理梯度，可以 **增大学习率**，通常将学习率乘以 $\text{batch\_size}$。
- 现代深度学习框架（如 TensorFlow 和 PyTorch）通常已经对梯度进行了归一化，因此无需手动调整学习率。

---

##### **2. 深度学习框架中提供了哪些损失函数和初始化方法？**

**答：**

###### **常见损失函数：**

- **回归任务**：
  - 均方误差损失（Mean Squared Error, MSE）
  - 平均绝对误差损失（Mean Absolute Error, MAE）
  - Huber 损失（Huber Loss）
  - 对数余弦损失（Log-Cosh Loss）
  
- **分类任务**：
  - 交叉熵损失（Cross-Entropy Loss）
  - 二分类交叉熵损失（Binary Cross-Entropy Loss）
  - Kullback-Leibler 散度（KL Divergence）

- **其他**：
  - 自定义损失函数

###### **常见初始化方法：**

- 随机初始化（Random Initialization）
- **标准初始化方法**：
  - Xavier 初始化（Glorot Initialization）
  - He 初始化（Kaiming Initialization）
  - 均匀分布初始化（Uniform Distribution）
  - 正态分布初始化（Normal Distribution）
- 零初始化和常数初始化（Zero and Constant Initialization）

###### **Huber 损失公式：**

$$
l(y, y') =
\begin{cases} 
|y - y'| - \frac{\sigma}{2}, & \text{if } |y - y'| > \sigma \\ 
\frac{1}{2\sigma}(y - y')^2, & \text{otherwise}.
\end{cases}
$$

在实现时，需要设定超参数 $\sigma$，用以控制线性和二次损失之间的切换点。框架通常会提供默认值，同时允许用户自定义。

---

##### **3. 如何访问线性回归的梯度？**

**答：**

在深度学习框架中，线性回归的梯度可以通过自动微分工具（如 TensorFlow 的 `tf.GradientTape` 或 PyTorch 的 `autograd`）进行计算。

##### 示例代码如下：

##### **PyTorch 示例：**

```python
import torch
import torch.nn as nn

# 定义模型
model = nn.Linear(1, 1)
criterion = nn.MSELoss()

# 输入和标签
x = torch.tensor([[1.0], [2.0]], requires_grad=True)
y = torch.tensor([[2.0], [4.0]])

# 前向传播
output = model(x)
loss = criterion(output, y)

# 反向传播
loss.backward()

# 访问梯度
for param in model.parameters():
    print(param.grad)